In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns 
import warnings
import itertools
warnings.filterwarnings('ignore')
from prophet import Prophet
from sklearn.metrics import mean_absolute_percentage_error as smape
from datetime import datetime
import itertools
import pyodbc
from dotenv.main import load_dotenv
import os

In [3]:
df = pd.read_csv("D:/PT Bukit Asam/Document 2024/Script Python/DM_production_transaction_Mine_Operation_202410210833.csv")
df

,Tanggal,Bulan,Tahun,Jenis,Tipe,Contractor Name,Area,Desc 2 Name,Loc Src Area Name,Loc Src Label,...,Calories Value,Range Calories Value,Product Type Name,loc_src_layers_group_name,Tonase,Jarak Angkut,Ton Jarak,Satuan,last_updated,Updated_Date
0,2019-01-03,1,2019,Real,JS,None,None,None,Banko Barat,Pit 1,...,None,None,Waste,None,12225.674452,2.141749,26184.322271,Ton,2019-01-31 23:59:59,2024-11-26 09:08:23.470
1,2019-01-03,1,2019,Real,JS,None,None,None,Banko Barat,Pit 1 Utara,...,None,None,Waste,None,45146.096314,4.283497,96691.592749,Ton,2019-01-31 23:59:59,2024-11-26 09:08:23.470
2,2019-01-03,1,2019,Real,JS,None,None,None,Banko Barat,Pit 3 Timur,...,None,None,Waste,None,22045.238319,2.141749,47215.360344,Ton,2019-01-31 23:59:59,2024-11-26 09:08:23.500
3,2019-01-03,1,2019,Real,JS,None,None,None,TAL,TAL Barat,...,5000,None,Mine Brand,None,7041.420972,2.914629,20523.129944,Ton,2019-01-31 23:59:59,2024-11-26 09:08:19.343
4,2019-01-03,1,2019,Real,JS,None,None,None,TAL,TAL Taman,...,None,None,Waste,None,12478.056137,4.283497,26724.860416,Ton,2019-01-31 23:59:59,2024-11-26 09:08:23.500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
254058,2019-07-03,7,2019,Plan,RKAP,None,None,None,TAL,TAL TSBC-Townsite,...,None,None,Waste,None,43709.677419,3.100000,135500.000000,Ton,2019-07-31 23:59:59,2024-11-26 09:08:26.267
254059,2020-07-24,7,2020,Plan,RKAP,None,None,None,Banko Barat,Pit 1 Utara,...,5100,None,Mine Brand,None,806.451613,2.400000,1935.483871,Ton,2020-07-31 23:59:59,2024-11-26 09:08:25.110
254060,2021-09-20,9,2021,Plan,Usuai,None,None,None,TAL,TAL-TSBC,...,None,None,Waste,None,35500.000000,3.750000,133125.000000,Ton,2021-09-30 23:59:59,2024-11-26 09:08:26.860
254061,2023-11-21,11,2023,Plan,RKAP,PT PAMA PERSADA NUSANTARA,BT-B Pit SJS,PT PAMA PERSADA NUSANTARA,Banko Tengah,BT-B Pit SJS,...,4500,None,Mine Brand,IUP Banko Tengah Blok A,4866.666667,3.000000,14600.000000,Ton,2023-11-30 23:59:59,2024-11-26 09:12:23.827


In [4]:
#  Preprocessing untuk mengambil coal dengan product type = Mine Brand dan Market Brand
df_temp_coal_real = df[df['Product Type Name'].isin(['Mine Brand','Market Brand'])].copy()

In [5]:
# preprocessing, mengambil nilai real tanpa plan (RKAP dan Rakor)
df_temp_coal_real = df_temp_coal_real[df_temp_coal_real['Jenis']=='Real']

In [6]:
# df produksi untuk all location, jadi per-unit lokasi akan digunakan setelah ini
df_all_location = df_temp_coal_real.groupby(['Tanggal'])['Tonase'].sum().reset_index()

In [7]:
# Mengubah tipe data tanggal menjadi datetime
df_all_location['Tanggal'] = pd.to_datetime(df_all_location['Tanggal'])

In [8]:
# Data yang diambil dari tanggl 1 januari 2020 (Sesuai Kesepakatan dengan Pihak PTBA)
df_prep_Date = df_all_location[df_all_location['Tanggal'] >= '2020-01-01']


In [9]:
# reset index berdasrkan tanggal terlama hingga sekarang
df_prep_Date_index = df_prep_Date.reset_index().sort_values(by=['Tanggal'])


In [10]:
# Kolom yang digunakan hanya "Tanggal dan Tonase", 
# dikarenakan tidak ada fitur lain yang berkaitan dengan Tonase maka kolom lain akan diabaikan
df_prep_Date_index = df_prep_Date_index[['Tanggal','Tonase']]

In [11]:
# Tidak Menyertakan Nilai 0 Dan null
temp_df_coal_all_area =  df_prep_Date_index[df_prep_Date_index['Tonase'] != 0].reset_index(drop=True).sort_values(by = ['Tanggal'])

In [12]:
# memastikan data <= tanggal hari ini
coal_all_area = temp_df_coal_all_area[temp_df_coal_all_area['Tanggal'] <= pd.to_datetime('today')]

In [13]:
# Uji coba Prophet mengubah nama kolom, menyesuaikan dengan model Prophet
coal_all_area.columns=['ds','y']

In [14]:
# Fungsi SMAPE
def smape(actual, predicted):
    return 100 * np.mean(2 * np.abs(predicted - actual) / (np.abs(predicted) + np.abs(actual)))

# Data Preparation (Anda bisa menyesuaikan dengan dataset Anda)
train_size = int(0.9 * len(coal_all_area))
train_data = coal_all_area[:train_size]
test_data = coal_all_area[train_size:]

# Parameter tuning grid
changepoint_prior_scales = [0.01, 0.05]
seasonality_prior_scales = [0.5, 1]
n_changepoints = [ 50, 100]
seasonality_modes = ['multiplicative']

# Kombinasi parameter
param_combinations = list(itertools.product(
    changepoint_prior_scales,
    seasonality_prior_scales,
    n_changepoints,
    seasonality_modes
))

# Variabel untuk menyimpan hasil tuning terbaik
best_smape = float('inf')
best_params = None

# Looping untuk setiap kombinasi parameter
for changepoint_prior_scale, seasonality_prior_scale, n_changepoints, seasonality_mode in param_combinations:
    
    # Inisialisasi model Prophet dengan parameter tuning
    model = Prophet(
        yearly_seasonality=True, 
        weekly_seasonality=False, 
        daily_seasonality=True,
        seasonality_mode=seasonality_mode,
        changepoint_prior_scale=changepoint_prior_scale,
        seasonality_prior_scale=seasonality_prior_scale,
        growth="linear",
        n_changepoints=n_changepoints,
        interval_width=0.95
    )
    
    # Menambahkan hari libur dan musiman tambahan jika diperlukan
    model.add_country_holidays(country_name='Indonesia')
    model.add_seasonality(name='monthly', period=30.5, fourier_order=5)

    # Melatih model dengan data train
    model.fit(train_data)

    # Membuat dataframe masa depan yang mencakup periode data test
    future = model.make_future_dataframe(periods=len(test_data), freq='D')

    # Melakukan prediksi
    forecast = model.predict(future)

    # Mengambil prediksi yang sesuai dengan data test
    forecast_test = forecast[-len(test_data):]

    # Menggabungkan data asli dengan prediksi
    test_data.loc[:, 'yhat'] = forecast_test['yhat'].values

    # Menghitung SMAPE untuk data test
    smape_value = smape(test_data['y'], test_data['yhat'])
    
    # Output hasil SMAPE
    print("PRODUCTION ALL AREA")
    print(f"SMAPE dengan changepoint_prior_scale={changepoint_prior_scale}, "
          f"seasonality_prior_scale={seasonality_prior_scale}, n_changepoints={n_changepoints}, "
          f"seasonality_mode={seasonality_mode}: {smape_value:.2f}%")

    # Menyimpan kombinasi parameter terbaik
    if smape_value < best_smape:
        best_smape = smape_value
        best_params = (changepoint_prior_scale, seasonality_prior_scale, n_changepoints, seasonality_mode)

# Menampilkan parameter terbaik
print(f"\nParameter terbaik: changepoint_prior_scale={best_params[0]}, "
      f"seasonality_prior_scale={best_params[1]}, n_changepoints={best_params[2]}, "
      f"seasonality_mode={best_params[3]}")
print(f"SMAPE terbaik: {best_smape:.2f}%")

# Perform cross-validation on the best model
best_model = Prophet(
    yearly_seasonality=True,
    weekly_seasonality=False,
    daily_seasonality=True,
    seasonality_mode=best_params[3],
    changepoint_prior_scale=best_params[0],
    seasonality_prior_scale=best_params[1],
    growth="linear",
    n_changepoints=best_params[2],
    interval_width=0.95
)
best_model.add_country_holidays(country_name='Indonesia')
best_model.add_seasonality(name='monthly', period=30.5, fourier_order=5)
best_model.fit(train_data)

# Latih model dengan seluruh data menggunakan parameter terbaik
final_model = Prophet(
    yearly_seasonality=True,
    weekly_seasonality=False,
    daily_seasonality=True,
    seasonality_mode=best_params[3],
    changepoint_prior_scale=best_params[0],
    seasonality_prior_scale=best_params[1],
    growth="linear",
    n_changepoints=best_params[2],
    interval_width=0.95
)
final_model.add_country_holidays(country_name='Indonesia')
final_model.add_seasonality(name='monthly', period=30.5, fourier_order=5)

# Melatih model dengan seluruh data
final_model.fit(coal_all_area)

# Membuat dataframe masa depan untuk 6 bulan ke depan
future = final_model.make_future_dataframe(periods=300, freq='d')

# Melakukan prediksi
forecast_final = final_model.predict(future)

# Menampilkan hasil prediksi
# print(forecast_final[['ds', 'yhat', 'yhat_lower', 'yhat_upper']])

# Menyimpan hasil prediksi ke dalam DataFrame
forecast_all_area = forecast_final[['ds', 'yhat']]
forecast_all_area.columns = ['Date', 'Forecast']


09:59:57 - cmdstanpy - INFO - Chain [1] start processing
09:59:58 - cmdstanpy - INFO - Chain [1] done processing


PRODUCTION ALL AREA
SMAPE dengan changepoint_prior_scale=0.01, seasonality_prior_scale=0.5, n_changepoints=50, seasonality_mode=multiplicative: 19.55%


09:59:58 - cmdstanpy - INFO - Chain [1] start processing
09:59:58 - cmdstanpy - INFO - Chain [1] done processing


PRODUCTION ALL AREA
SMAPE dengan changepoint_prior_scale=0.01, seasonality_prior_scale=0.5, n_changepoints=100, seasonality_mode=multiplicative: 19.74%


09:59:59 - cmdstanpy - INFO - Chain [1] start processing
09:59:59 - cmdstanpy - INFO - Chain [1] done processing


...

10:00:02 - cmdstanpy - INFO - Chain [1] start processing
10:00:02 - cmdstanpy - INFO - Chain [1] done processing


PRODUCTION ALL AREA
SMAPE dengan changepoint_prior_scale=0.05, seasonality_prior_scale=1, n_changepoints=50, seasonality_mode=multiplicative: 19.20%


10:00:03 - cmdstanpy - INFO - Chain [1] start processing
10:00:03 - cmdstanpy - INFO - Chain [1] done processing


PRODUCTION ALL AREA
SMAPE dengan changepoint_prior_scale=0.05, seasonality_prior_scale=1, n_changepoints=100, seasonality_mode=multiplicative: 19.42%

Parameter terbaik: changepoint_prior_scale=0.05, seasonality_prior_scale=1, n_changepoints=50, seasonality_mode=multiplicative
SMAPE terbaik: 19.20%


10:00:04 - cmdstanpy - INFO - Chain [1] start processing
10:00:04 - cmdstanpy - INFO - Chain [1] done processing
10:00:04 - cmdstanpy - INFO - Chain [1] start processing
10:00:04 - cmdstanpy - INFO - Chain [1] done processing


In [15]:
# Pastikan kolom 'Date' dalam format datetime
forecast_all_area['Date'] = pd.to_datetime(forecast_all_area['Date']).dt.date
# Menambahkan data untuk 'Forecast'
forecast_rows = pd.DataFrame({
    'Tanggal': forecast_all_area['Date'],
    'Tonase': forecast_all_area['Forecast'].round(3),
    'Jenis': 'Forecast',
    'Kategori' : 'Coal',
    'Loc Src Area Name' : 'All Area'
})

df_production_actual = pd.DataFrame({
    'Tanggal': pd.to_datetime(coal_all_area['ds']).dt.date,
    'Tonase': coal_all_area['y'].round(3),
    'Jenis': 'Actual',
    'Kategori' : 'Coal',
    'Loc Src Area Name' : 'All Area'
})
current_time = datetime.today().date()
best_smape_rounded = round(best_smape, 3)
smape_all = pd.DataFrame({
    'Tanggal' : current_time,
    'Tonase' : [best_smape_rounded],
    'Jenis' :['Smape'],
    'Kategori' : ['Coal'],
    'Loc Src Area Name' : ['All Area']
})

# Menggabungkan semua DataFrame
result_df_production = pd.concat([df_production_actual,forecast_rows, smape_all], ignore_index=True)
# result_df_production


In [16]:
# Filter Lokasi Coal pada MTB
df_temp_coal_MTB = df_temp_coal_real[df_temp_coal_real['Loc Src Area Name']=='MTB']

In [17]:
# 
temp_coal_MTB = df_temp_coal_MTB.groupby(['Tanggal'])['Tonase'].sum().reset_index()

In [18]:
temp_coal_MTB['Tanggal'] = pd.to_datetime(temp_coal_MTB['Tanggal'])

In [19]:
# Data yang diambil dari tanggl 1 januari 2020 (Sesuai Kesepakatan dengan Pihak PTBA)
df_prep_Date_MTB = temp_coal_MTB[temp_coal_MTB['Tanggal'] >= '2020-01-01']

In [20]:
# reset index
df_prep_Date_index_MTB = df_prep_Date_MTB.reset_index().sort_values(by=['Tanggal'])
df_prep_Date_index_MTB = df_prep_Date_index_MTB[['Tanggal','Tonase']]

In [21]:
# Tidak Menyertakan Nilai 0 Dan null
temp_df_coal_MTB =  df_prep_Date_index_MTB[df_prep_Date_index_MTB['Tonase'] != 0].reset_index(drop=True).sort_values(by = ['Tanggal'])


In [22]:
coal_all_area_MTB = temp_df_coal_MTB[temp_df_coal_MTB['Tanggal'] <= pd.to_datetime('today')]

In [23]:
coal_all_area_MTB.columns=['ds','y']

In [24]:
# Fungsi SMAPE
def smape(actual, predicted):
    return 100 * np.mean(2 * np.abs(predicted - actual) / (np.abs(predicted) + np.abs(actual)))

# Data Preparation (Anda bisa menyesuaikan dengan dataset Anda)
train_size = int(0.9 * len(coal_all_area_MTB))
train_data = coal_all_area_MTB[:train_size]
test_data = coal_all_area_MTB[train_size:]

# Parameter tuning grid
changepoint_prior_scales = [0.1, 0.2]
seasonality_prior_scales = [2,5]
n_changepoints = [100, 150]
seasonality_modes = ['multiplicative']

# Kombinasi parameter
param_combinations = list(itertools.product(
    changepoint_prior_scales,
    seasonality_prior_scales,
    n_changepoints,
    seasonality_modes
))

# Variabel untuk menyimpan hasil tuning terbaik
best_smape = float('inf')
best_params = None

# Looping untuk setiap kombinasi parameter
for changepoint_prior_scale, seasonality_prior_scale, n_changepoints, seasonality_mode in param_combinations:
    
    # Inisialisasi model Prophet dengan parameter tuning
    model = Prophet(
        yearly_seasonality=True, 
        weekly_seasonality=False, 
        daily_seasonality=True,
        seasonality_mode=seasonality_mode,
        changepoint_prior_scale=changepoint_prior_scale,
        seasonality_prior_scale=seasonality_prior_scale,
        growth="linear",
        n_changepoints=n_changepoints,
        interval_width=0.95
    )
    
    # Menambahkan hari libur dan musiman tambahan jika diperlukan
    model.add_country_holidays(country_name='Indonesia')
    model.add_seasonality(name='monthly', period=30.5, fourier_order=5)

    # Melatih model dengan data train
    model.fit(train_data)

    # Membuat dataframe masa depan yang mencakup periode data test
    future = model.make_future_dataframe(periods=len(test_data), freq='D')

    # Melakukan prediksi
    forecast = model.predict(future)

    # Mengambil prediksi yang sesuai dengan data test
    forecast_test = forecast[-len(test_data):]

    # Menggabungkan data asli dengan prediksi
    test_data.loc[:, 'yhat'] = forecast_test['yhat'].values

    # Menghitung SMAPE untuk data test
    smape_value = smape(test_data['y'], test_data['yhat'])
    
    # Output hasil SMAPE
    print("PRODUCTION COAL MTB")
    print(f"SMAPE dengan changepoint_prior_scale={changepoint_prior_scale}, "
          f"seasonality_prior_scale={seasonality_prior_scale}, n_changepoints={n_changepoints}, "
          f"seasonality_mode={seasonality_mode}: {smape_value:.2f}%")

    # Menyimpan kombinasi parameter terbaik
    if smape_value < best_smape:
        best_smape = smape_value
        best_params = (changepoint_prior_scale, seasonality_prior_scale, n_changepoints, seasonality_mode)

# Menampilkan parameter terbaik
print(f"\nParameter terbaik: changepoint_prior_scale={best_params[0]}, "
      f"seasonality_prior_scale={best_params[1]}, n_changepoints={best_params[2]}, "
      f"seasonality_mode={best_params[3]}")
print(f"SMAPE terbaik: {best_smape:.2f}%")

# Perform cross-validation on the best model
best_model= Prophet(
    yearly_seasonality=True,
    weekly_seasonality=False,
    daily_seasonality=True,
    seasonality_mode=best_params[3],
    changepoint_prior_scale=best_params[0],
    seasonality_prior_scale=best_params[1],
    growth="linear",
    n_changepoints=best_params[2],
    interval_width=0.95
)
# Latih model dengan seluruh data menggunakan parameter terbaik
final_model = Prophet(
    yearly_seasonality=True,
    weekly_seasonality=False,
    daily_seasonality=True,
    seasonality_mode=best_params[3],
    changepoint_prior_scale=best_params[0],
    seasonality_prior_scale=best_params[1],
    growth="linear",
    n_changepoints=best_params[2],
    interval_width=0.95
)
final_model.add_country_holidays(country_name='Indonesia')
final_model.add_seasonality(name='monthly', period=30.5, fourier_order=5)

# Melatih model dengan seluruh data
final_model.fit(coal_all_area_MTB)

# Membuat dataframe masa depan untuk 6 bulan ke depan
future = final_model.make_future_dataframe(periods=300, freq='d')

# Melakukan prediksi
forecast_final = final_model.predict(future)

# Menyimpan hasil prediksi ke dalam DataFrame
forecast_all_area_MTB= forecast_final[['ds', 'yhat']]
forecast_all_area_MTB.columns = ['Date', 'Forecast']


10:00:05 - cmdstanpy - INFO - Chain [1] start processing
10:00:09 - cmdstanpy - INFO - Chain [1] done processing


PRODUCTION COAL MTB
SMAPE dengan changepoint_prior_scale=0.1, seasonality_prior_scale=2, n_changepoints=100, seasonality_mode=multiplicative: 19.94%


10:00:09 - cmdstanpy - INFO - Chain [1] start processing
10:00:13 - cmdstanpy - INFO - Chain [1] done processing


PRODUCTION COAL MTB
SMAPE dengan changepoint_prior_scale=0.1, seasonality_prior_scale=2, n_changepoints=150, seasonality_mode=multiplicative: 19.96%


10:00:14 - cmdstanpy - INFO - Chain [1] start processing
10:00:18 - cmdstanpy - INFO - Chain [1] done processing


...

10:00:34 - cmdstanpy - INFO - Chain [1] start processing
10:00:38 - cmdstanpy - INFO - Chain [1] done processing


PRODUCTION COAL MTB
SMAPE dengan changepoint_prior_scale=0.2, seasonality_prior_scale=5, n_changepoints=100, seasonality_mode=multiplicative: 19.75%


10:00:38 - cmdstanpy - INFO - Chain [1] start processing
10:00:42 - cmdstanpy - INFO - Chain [1] done processing


PRODUCTION COAL MTB
SMAPE dengan changepoint_prior_scale=0.2, seasonality_prior_scale=5, n_changepoints=150, seasonality_mode=multiplicative: 19.74%

Parameter terbaik: changepoint_prior_scale=0.2, seasonality_prior_scale=2, n_changepoints=150, seasonality_mode=multiplicative
SMAPE terbaik: 19.72%


10:00:43 - cmdstanpy - INFO - Chain [1] start processing
10:00:48 - cmdstanpy - INFO - Chain [1] done processing


In [25]:
# Pastikan kolom 'Date' dalam format datetime
forecast_all_area_MTB['Date'] = pd.to_datetime(forecast_all_area_MTB['Date']).dt.date
# Menambahkan data untuk 'Forecast'
forecast_rows = pd.DataFrame({
    'Tanggal': forecast_all_area_MTB['Date'],
    'Tonase' :  forecast_all_area_MTB['Forecast'].round(3),
    'Jenis': 'Forecast',
    'Kategori' : 'Coal',
    'Loc Src Area Name' : 'MTB'
})

df_production_actual_MTB = pd.DataFrame({
    'Tanggal': pd.to_datetime(coal_all_area_MTB['ds']).dt.date,
    'Tonase': coal_all_area_MTB['y'].round(3),
    'Jenis': 'Actual',
    'Kategori' : 'Coal',
    'Loc Src Area Name' : 'MTB'
})
current_time = datetime.today().date()
best_smape_rounded = round(best_smape, 3)
smape_MTB = pd.DataFrame({
    'Tanggal' : current_time,
    'Tonase' : [best_smape_rounded],
    'Jenis' :['Smape'],
    'Kategori' : ['Coal'],
    'Loc Src Area Name' : ['MTB']
})
# Menggabungkan semua DataFrame
result_df_MTB = pd.concat([df_production_actual_MTB,forecast_rows,smape_MTB], ignore_index=True)
# result_df_MTB

---

In [26]:
# MTB
df_temp_coal_TAL = df_temp_coal_real[df_temp_coal_real['Loc Src Area Name']=='TAL']


In [27]:
temp_coal_TAL = df_temp_coal_TAL.groupby(['Tanggal'])['Tonase'].sum().reset_index()

In [28]:
temp_coal_TAL['Tanggal'] = pd.to_datetime(temp_coal_TAL['Tanggal'])

In [29]:
# Data yang diambil dari tanggl 1 januari 2020 (Sesuai Kesepakatan dengan Pihak PTBA)
df_prep_Date_TAL = temp_coal_TAL[temp_coal_TAL['Tanggal'] >= '2020-01-01']

In [30]:
# reset index
df_prep_Date_index_TAL = df_prep_Date_TAL.reset_index().sort_values(by=['Tanggal'])
df_prep_Date_index_TAL = df_prep_Date_index_TAL[['Tanggal','Tonase']]

In [31]:
# Tidak Menyertakan Nilai 0 Dan null
temp_df_coal_TAL =  df_prep_Date_index_TAL[df_prep_Date_index_TAL['Tonase'] != 0].reset_index(drop=True).sort_values(by = ['Tanggal'])

In [32]:
coal_all_area_TAL = temp_df_coal_TAL[temp_df_coal_TAL['Tanggal'] <= pd.to_datetime('today')]

In [33]:
coal_all_area_TAL.columns=['ds','y']

In [34]:
# Fungsi SMAPE
def smape(actual, predicted):
    return 100 * np.mean(2 * np.abs(predicted - actual) / (np.abs(predicted) + np.abs(actual)))

# Data Preparation (Anda bisa menyesuaikan dengan dataset Anda)
train_size = int(0.9 * len(coal_all_area_TAL))
train_data = coal_all_area_TAL[:train_size]
test_data = coal_all_area_TAL[train_size:]

# Parameter tuning grid
changepoint_prior_scales = [ 0.1, 0.2]
seasonality_prior_scales = [0.5, 1]
n_changepoints = [ 50, 100]
seasonality_modes = ['multiplicative']

# Kombinasi parameter
param_combinations = list(itertools.product(
    changepoint_prior_scales,
    seasonality_prior_scales,
    n_changepoints,
    seasonality_modes
))

# Variabel untuk menyimpan hasil tuning terbaik
best_smape = float('inf')
best_params = None

# Looping untuk setiap kombinasi parameter
for changepoint_prior_scale, seasonality_prior_scale, n_changepoints, seasonality_mode in param_combinations:
    
    # Inisialisasi model Prophet dengan parameter tuning
    model = Prophet(
        yearly_seasonality=True, 
        weekly_seasonality=True, 
        daily_seasonality=True,
        seasonality_mode=seasonality_mode,
        changepoint_prior_scale=changepoint_prior_scale,
        seasonality_prior_scale=seasonality_prior_scale,
        growth="flat",
        n_changepoints=n_changepoints,
        interval_width=0.95

    )
    
    # Menambahkan hari libur dan musiman tambahan jika diperlukan
    model.add_country_holidays(country_name='Indonesia')
    model.add_seasonality(name='monthly', period=30.5, fourier_order=5)

    # Melatih model dengan data train
    model.fit(train_data)

    # Membuat dataframe masa depan yang mencakup periode data test
    future = model.make_future_dataframe(periods=len(test_data), freq='D')

    # Melakukan prediksi
    forecast = model.predict(future)

    # Mengambil prediksi yang sesuai dengan data test
    forecast_test = forecast[-len(test_data):]

    # Menggabungkan data asli dengan prediksi
    test_data.loc[:, 'yhat'] = forecast_test['yhat'].values

    # Menghitung SMAPE untuk data test
    smape_value = smape(test_data['y'], test_data['yhat'])
    
    # Output hasil SMAPE
    print("PRODUCTION COAL TAL")
    print(f"SMAPE dengan changepoint_prior_scale={changepoint_prior_scale}, "
          f"seasonality_prior_scale={seasonality_prior_scale}, n_changepoints={n_changepoints}, "
          f"seasonality_mode={seasonality_mode}: {smape_value:.2f}%")

    # Menyimpan kombinasi parameter terbaik
    if smape_value < best_smape:
        best_smape = smape_value
        best_params = (changepoint_prior_scale, seasonality_prior_scale, n_changepoints, seasonality_mode)

# Menampilkan parameter terbaik
print(f"\nParameter terbaik: changepoint_prior_scale={best_params[0]}, "
      f"seasonality_prior_scale={best_params[1]}, n_changepoints={best_params[2]}, "
      f"seasonality_mode={best_params[3]}")
print(f"SMAPE terbaik: {best_smape:.2f}%")

# Perform cross-validation on the best model
best_model = Prophet(
    yearly_seasonality=True,
    weekly_seasonality=False,
    daily_seasonality=True,
    seasonality_mode=best_params[3],
    changepoint_prior_scale=best_params[0],
    seasonality_prior_scale=best_params[1],
    growth="flat",
    n_changepoints=best_params[2],
    interval_width=0.95
)
best_model.add_country_holidays(country_name='Indonesia')
best_model.add_seasonality(name='monthly', period=30.5, fourier_order=5)
best_model.fit(train_data)

# Latih model dengan seluruh data menggunakan parameter terbaik
final_model = Prophet(
    yearly_seasonality=True,
    weekly_seasonality=True,
    daily_seasonality=True,
    seasonality_mode=best_params[3],
    changepoint_prior_scale=best_params[0],
    seasonality_prior_scale=best_params[1],
    growth="flat",
    n_changepoints=best_params[2],
    interval_width=0.95
)
final_model.add_country_holidays(country_name='Indonesia')
final_model.add_seasonality(name='monthly', period=30.5, fourier_order=5)

# Melatih model dengan seluruh data
final_model.fit(coal_all_area_TAL)

# Membuat dataframe masa depan untuk 6 bulan ke depan
future = final_model.make_future_dataframe(periods=300, freq='d')

# Melakukan prediksi
forecast_final = final_model.predict(future)

# Menampilkan hasil prediksi
# print(forecast_final[['ds', 'yhat', 'yhat_lower', 'yhat_upper']])

# Menyimpan hasil prediksi ke dalam DataFrame
forecast_all_area_TAL= forecast_final[['ds', 'yhat']]
forecast_all_area_TAL.columns = ['Date', 'Forecast']


10:00:49 - cmdstanpy - INFO - Chain [1] start processing
10:00:49 - cmdstanpy - INFO - Chain [1] done processing


PRODUCTION COAL TAL
SMAPE dengan changepoint_prior_scale=0.1, seasonality_prior_scale=0.5, n_changepoints=50, seasonality_mode=multiplicative: 21.58%


10:00:50 - cmdstanpy - INFO - Chain [1] start processing
10:00:50 - cmdstanpy - INFO - Chain [1] done processing


PRODUCTION COAL TAL
SMAPE dengan changepoint_prior_scale=0.1, seasonality_prior_scale=0.5, n_changepoints=100, seasonality_mode=multiplicative: 21.58%


10:00:50 - cmdstanpy - INFO - Chain [1] start processing
10:00:50 - cmdstanpy - INFO - Chain [1] done processing


...

10:00:53 - cmdstanpy - INFO - Chain [1] start processing
10:00:53 - cmdstanpy - INFO - Chain [1] done processing


PRODUCTION COAL TAL
SMAPE dengan changepoint_prior_scale=0.2, seasonality_prior_scale=1, n_changepoints=50, seasonality_mode=multiplicative: 21.58%


10:00:53 - cmdstanpy - INFO - Chain [1] start processing
10:00:54 - cmdstanpy - INFO - Chain [1] done processing


PRODUCTION COAL TAL
SMAPE dengan changepoint_prior_scale=0.2, seasonality_prior_scale=1, n_changepoints=100, seasonality_mode=multiplicative: 21.58%

Parameter terbaik: changepoint_prior_scale=0.1, seasonality_prior_scale=0.5, n_changepoints=100, seasonality_mode=multiplicative
SMAPE terbaik: 21.58%


10:00:54 - cmdstanpy - INFO - Chain [1] start processing
10:00:54 - cmdstanpy - INFO - Chain [1] done processing
10:00:55 - cmdstanpy - INFO - Chain [1] start processing
10:00:55 - cmdstanpy - INFO - Chain [1] done processing


In [35]:
# Pastikan kolom 'Date' dalam format datetime
forecast_all_area_TAL['Date'] = pd.to_datetime(forecast_all_area_TAL['Date']).dt.date
# Menambahkan data untuk 'Forecast'
forecast_rows = pd.DataFrame({
    'Tanggal': forecast_all_area_TAL['Date'],
    'Tonase': forecast_all_area_TAL['Forecast'].round(3),
    'Jenis': 'Forecast',
    'Kategori' : 'Coal',
    'Loc Src Area Name' : 'TAL'
})

df_production_actual_TAL = pd.DataFrame({
    'Tanggal': pd.to_datetime(coal_all_area_TAL['ds']).dt.date,
    'Tonase': coal_all_area_TAL['y'].round(3),
    'Jenis': 'Actual',
    'Kategori' : 'Coal',
    'Loc Src Area Name' : 'TAL'
})
current_time = datetime.today().date()
best_smape_rounded = round(best_smape, 3)
smape_TAL = pd.DataFrame({
    'Tanggal' : current_time,
    'Tonase' : [best_smape_rounded],
    'Jenis' :['Smape'],
    'Kategori' : ['Coal'],
    'Loc Src Area Name' : ['TAL']
})

# Menggabungkan semua DataFrame
result_df_TAL = pd.concat([df_production_actual_TAL,forecast_rows, smape_TAL], ignore_index=True)
# result_df_TAL

In [36]:
# Banko Barat
df_temp_coal_BB = df_temp_coal_real[df_temp_coal_real['Loc Src Area Name']=='Banko Barat']

In [37]:
df_temp_coal_BB = df_temp_coal_BB.groupby(['Tanggal'])['Tonase'].sum().reset_index()

In [38]:
df_temp_coal_BB['Tanggal'] = pd.to_datetime(df_temp_coal_BB['Tanggal'])

In [39]:
# Data yang diambil dari tanggl 1 januari 2020 (Sesuai Kesepakatan dengan Pihak PTBA)
df_prep_Date_BB = df_temp_coal_BB[df_temp_coal_BB['Tanggal'] >= '2020-01-01']

In [40]:
# reset index
df_prep_Date_index_BB = df_prep_Date_BB.reset_index().sort_values(by=['Tanggal'])
df_prep_Date_index_BB = df_prep_Date_index_BB[['Tanggal','Tonase']]


In [41]:
# Tidak Menyertakan Nilai 0 Dan null
temp_df_coal_BB =  df_prep_Date_index_BB[df_prep_Date_index_BB['Tonase'] != 0].reset_index(drop=True).sort_values(by = ['Tanggal'])

In [42]:
coal_all_area_BB = temp_df_coal_BB[temp_df_coal_BB['Tanggal'] <= pd.to_datetime('today')]

In [43]:
coal_all_area_BB.columns=['ds','y']

In [44]:
# Fungsi SMAPE
def smape(actual, predicted):
    return 100 * np.mean(2 * np.abs(predicted - actual) / (np.abs(predicted) + np.abs(actual)))

# Data Preparation (Anda bisa menyesuaikan dengan dataset Anda)
train_size = int(0.9 * len(coal_all_area_BB))
train_data = coal_all_area_BB[:train_size]
test_data = coal_all_area_BB[train_size:]

# Parameter tuning grid
changepoint_prior_scales = [0.2]
seasonality_prior_scales = [0.5, 1,2]
n_changepoints = [ 100, 150]
seasonality_modes = ['multiplicative']

# Kombinasi parameter
param_combinations = list(itertools.product(
    changepoint_prior_scales,
    seasonality_prior_scales,
    n_changepoints,
    seasonality_modes
))

# Variabel untuk menyimpan hasil tuning terbaik
best_smape = float('inf')
best_params = None

# Looping untuk setiap kombinasi parameter
for changepoint_prior_scale, seasonality_prior_scale, n_changepoints, seasonality_mode in param_combinations:
    
    # Inisialisasi model Prophet dengan parameter tuning
    model = Prophet(
        yearly_seasonality=True, 
        weekly_seasonality=False, 
        daily_seasonality=True,
        seasonality_mode=seasonality_mode,
        changepoint_prior_scale=changepoint_prior_scale,
        seasonality_prior_scale=seasonality_prior_scale,
        growth="linear",
        n_changepoints=n_changepoints,
        interval_width=0.95

    )
    
    # Menambahkan hari libur dan musiman tambahan jika diperlukan
    model.add_country_holidays(country_name='Indonesia')
    model.add_seasonality(name='monthly', period=30.5, fourier_order=5)

    # Melatih model dengan data train
    model.fit(train_data)

    # Membuat dataframe masa depan yang mencakup periode data test
    future = model.make_future_dataframe(periods=len(test_data), freq='D')

    # Melakukan prediksi
    forecast = model.predict(future)

    # Mengambil prediksi yang sesuai dengan data test
    forecast_test = forecast[-len(test_data):]

    # Menggabungkan data asli dengan prediksi
    test_data.loc[:, 'yhat'] = forecast_test['yhat'].values

    # Menghitung SMAPE untuk data test
    smape_value = smape(test_data['y'], test_data['yhat'])
    
    # Output hasil SMAPE
    print(f"SMAPE dengan changepoint_prior_scale={changepoint_prior_scale}, "
          f"seasonality_prior_scale={seasonality_prior_scale}, n_changepoints={n_changepoints}, "
          f"seasonality_mode={seasonality_mode}: {smape_value:.2f}%")

    # Menyimpan kombinasi parameter terbaik
    if smape_value < best_smape:
        best_smape = smape_value
        best_params = (changepoint_prior_scale, seasonality_prior_scale, n_changepoints, seasonality_mode)

# Menampilkan parameter terbaik
print(f"\nParameter terbaik: changepoint_prior_scale={best_params[0]}, "
      f"seasonality_prior_scale={best_params[1]}, n_changepoints={best_params[2]}, "
      f"seasonality_mode={best_params[3]}")
print(f"SMAPE terbaik: {best_smape:.2f}%")

# Perform cross-validation on the best model
best_model = Prophet(
    yearly_seasonality=True,
    weekly_seasonality=False,
    daily_seasonality=True,
    seasonality_mode=best_params[3],
    changepoint_prior_scale=best_params[0],
    seasonality_prior_scale=best_params[1],
    growth="linear",
    n_changepoints=best_params[2],
    interval_width=0.95
)
best_model.add_country_holidays(country_name='Indonesia')
best_model.add_seasonality(name='monthly', period=30.5, fourier_order=5)
best_model.fit(train_data)

# Latih model dengan seluruh data menggunakan parameter terbaik
final_model = Prophet(
    yearly_seasonality=True,
    weekly_seasonality=False,
    daily_seasonality=True,
    seasonality_mode=best_params[3],
    changepoint_prior_scale=best_params[0],
    seasonality_prior_scale=best_params[1],
    growth="linear",
    n_changepoints=best_params[2],
    interval_width=0.95
)
final_model.add_country_holidays(country_name='Indonesia')
final_model.add_seasonality(name='monthly', period=30.5, fourier_order=5)

# Melatih model dengan seluruh data
final_model.fit(coal_all_area_BB)

# Membuat dataframe masa depan untuk 6 bulan ke depan
future = final_model.make_future_dataframe(periods=300, freq='d')

# Melakukan prediksi
forecast_final = final_model.predict(future)

# Menampilkan hasil prediksi
# print(forecast_final[['ds', 'yhat', 'yhat_lower', 'yhat_upper']])

# Menyimpan hasil prediksi ke dalam DataFrame
forecast_all_area_BB= forecast_final[['ds', 'yhat']]
forecast_all_area_BB.columns = ['Date', 'Forecast']


10:00:55 - cmdstanpy - INFO - Chain [1] start processing
10:00:59 - cmdstanpy - INFO - Chain [1] done processing


SMAPE dengan changepoint_prior_scale=0.2, seasonality_prior_scale=0.5, n_changepoints=100, seasonality_mode=multiplicative: 32.90%


10:01:00 - cmdstanpy - INFO - Chain [1] start processing
10:01:04 - cmdstanpy - INFO - Chain [1] done processing


SMAPE dengan changepoint_prior_scale=0.2, seasonality_prior_scale=0.5, n_changepoints=150, seasonality_mode=multiplicative: 30.16%


10:01:05 - cmdstanpy - INFO - Chain [1] start processing
10:01:08 - cmdstanpy - INFO - Chain [1] done processing


...

10:01:14 - cmdstanpy - INFO - Chain [1] start processing
10:01:17 - cmdstanpy - INFO - Chain [1] done processing


SMAPE dengan changepoint_prior_scale=0.2, seasonality_prior_scale=2, n_changepoints=100, seasonality_mode=multiplicative: 40.01%


10:01:18 - cmdstanpy - INFO - Chain [1] start processing
10:01:22 - cmdstanpy - INFO - Chain [1] done processing


SMAPE dengan changepoint_prior_scale=0.2, seasonality_prior_scale=2, n_changepoints=150, seasonality_mode=multiplicative: 38.69%

Parameter terbaik: changepoint_prior_scale=0.2, seasonality_prior_scale=0.5, n_changepoints=150, seasonality_mode=multiplicative
SMAPE terbaik: 30.16%


10:01:22 - cmdstanpy - INFO - Chain [1] start processing
10:01:27 - cmdstanpy - INFO - Chain [1] done processing
10:01:27 - cmdstanpy - INFO - Chain [1] start processing
10:01:32 - cmdstanpy - INFO - Chain [1] done processing


In [45]:
# Pastikan kolom 'Date' dalam format datetime
forecast_all_area_BB['Date'] = pd.to_datetime(forecast_all_area_BB['Date']).dt.date
# Menambahkan data untuk 'Forecast'
forecast_rows = pd.DataFrame({
    'Tanggal': forecast_all_area_BB['Date'],
    'Tonase': forecast_all_area_BB['Forecast'].round(3),
    'Jenis': 'Forecast',
    'Kategori' : 'Coal',
    'Loc Src Area Name' : 'Banko Barat'
})

df_production_actual_BB = pd.DataFrame({
    'Tanggal': pd.to_datetime(coal_all_area_BB['ds']).dt.date,
    'Tonase': coal_all_area_BB['y'].round(3),
    'Jenis': 'Actual',
    'Kategori' : 'Coal',
    'Loc Src Area Name' : 'Banko Barat'
})
current_time = datetime.today().date()
best_smape_rounded = round(best_smape, 3)
smape_BB = pd.DataFrame({
    'Tanggal' : current_time,
    'Tonase' : [best_smape_rounded],
    'Jenis' :['Smape'],
    'Kategori' : ['Coal'],
    'Loc Src Area Name' : ['Banko Barat']
})

# Menggabungkan semua DataFrame
result_df_BB = pd.concat([df_production_actual_BB,forecast_rows, smape_BB], ignore_index=True)


In [46]:
#  Banko Tengah

In [47]:
# Banko Tengah
df_temp_coal_BT = df_temp_coal_real[df_temp_coal_real['Loc Src Area Name']=='Banko Tengah']


In [48]:
df_temp_coal_BT = df_temp_coal_BT.groupby(['Tanggal'])['Tonase'].sum().reset_index()

In [49]:
df_temp_coal_BT['Tanggal'] = pd.to_datetime(df_temp_coal_BT['Tanggal'])

In [50]:
# Data yang diambil dari tanggl 1 januari 2020 (Sesuai Kesepakatan dengan Pihak PTBA)
df_prep_Date_BT = df_temp_coal_BT[df_temp_coal_BT['Tanggal'] >= '2020-01-01']

In [51]:
# reset index
df_prep_Date_index_BT = df_prep_Date_BT.reset_index().sort_values(by=['Tanggal'])
df_prep_Date_index_BT = df_prep_Date_index_BT[['Tanggal','Tonase']]

In [52]:

# Tidak Menyertakan Nilai 0 Dan null
temp_df_coal_BT =  df_prep_Date_index_BT[df_prep_Date_index_BT['Tonase'] != 0].reset_index(drop=True).sort_values(by = ['Tanggal'])

In [53]:
coal_all_area_BT = temp_df_coal_BT[temp_df_coal_BT['Tanggal'] <= pd.to_datetime('today')]
coal_all_area_BT.columns=['ds','y']

In [54]:
# Fungsi SMAPE
def smape(actual, predicted):
    return 100 * np.mean(2 * np.abs(predicted - actual) / (np.abs(predicted) + np.abs(actual)))

# Data Preparation (Anda bisa menyesuaikan dengan dataset Anda)
train_size = int(0.9 * len(coal_all_area_BT))
train_data = coal_all_area_BT[:train_size]
test_data = coal_all_area_BT[train_size:]

# Parameter tuning grid
changepoint_prior_scales = [  0.2, 0.5]
seasonality_prior_scales = [0.5, 1,2]
n_changepoints = [ 100, 150]
seasonality_modes = ['multiplicative']

# Kombinasi parameter
param_combinations = list(itertools.product(
    changepoint_prior_scales,
    seasonality_prior_scales,
    n_changepoints,
    seasonality_modes
))

# Variabel untuk menyimpan hasil tuning terbaik
best_smape = float('inf')
best_params = None

# Looping untuk setiap kombinasi parameter
for changepoint_prior_scale, seasonality_prior_scale, n_changepoints, seasonality_mode in param_combinations:
    
    # Inisialisasi model Prophet dengan parameter tuning
    model = Prophet(
        yearly_seasonality=True, 
        weekly_seasonality=True, 
        daily_seasonality=True,
        seasonality_mode=seasonality_mode,
        changepoint_prior_scale=changepoint_prior_scale,
        seasonality_prior_scale=seasonality_prior_scale,
        growth="linear",
        n_changepoints=n_changepoints,
        interval_width=0.95

    )
    
    # Menambahkan hari libur dan musiman tambahan jika diperlukan
    model.add_country_holidays(country_name='Indonesia')
    model.add_seasonality(name='monthly', period=30.5, fourier_order=5)

    # Melatih model dengan data train
    model.fit(train_data)

    # Membuat dataframe masa depan yang mencakup periode data test
    future = model.make_future_dataframe(periods=len(test_data), freq='D')

    # Melakukan prediksi
    forecast = model.predict(future)

    # Mengambil prediksi yang sesuai dengan data test
    forecast_test = forecast[-len(test_data):]

    # Menggabungkan data asli dengan prediksi
    test_data.loc[:, 'yhat'] = forecast_test['yhat'].values

    # Menghitung SMAPE untuk data test
    smape_value = smape(test_data['y'], test_data['yhat'])
    
    # Output hasil SMAPE
    print(f"SMAPE dengan changepoint_prior_scale={changepoint_prior_scale}, "
          f"seasonality_prior_scale={seasonality_prior_scale}, n_changepoints={n_changepoints}, "
          f"seasonality_mode={seasonality_mode}: {smape_value:.2f}%")

    # Menyimpan kombinasi parameter terbaik
    if smape_value < best_smape:
        best_smape = smape_value
        best_params = (changepoint_prior_scale, seasonality_prior_scale, n_changepoints, seasonality_mode)

# Menampilkan parameter terbaik
print(f"\nParameter terbaik: changepoint_prior_scale={best_params[0]}, "
      f"seasonality_prior_scale={best_params[1]}, n_changepoints={best_params[2]}, "
      f"seasonality_mode={best_params[3]}")
print(f"SMAPE terbaik: {best_smape:.2f}%")

# Perform cross-validation on the best model
best_model = Prophet(
    yearly_seasonality=True,
    weekly_seasonality=False,
    daily_seasonality=True,
    seasonality_mode=best_params[3],
    changepoint_prior_scale=best_params[0],
    seasonality_prior_scale=best_params[1],
    growth="linear",
    n_changepoints=best_params[2],
    interval_width=0.95
)
best_model.add_country_holidays(country_name='Indonesia')
best_model.add_seasonality(name='monthly', period=30.5, fourier_order=5)
best_model.fit(train_data)
# Latih model dengan seluruh data menggunakan parameter terbaik
final_model = Prophet(
    yearly_seasonality=True,
    weekly_seasonality=True,
    daily_seasonality=True,
    seasonality_mode=best_params[3],
    changepoint_prior_scale=best_params[0],
    seasonality_prior_scale=best_params[1],
    growth="linear",
    n_changepoints=best_params[2],
    interval_width=0.95
)
final_model.add_country_holidays(country_name='Indonesia')
final_model.add_seasonality(name='monthly', period=30.5, fourier_order=5)

# Melatih model dengan seluruh data
final_model.fit(coal_all_area_BT)

# Membuat dataframe masa depan untuk 6 bulan ke depan
future = final_model.make_future_dataframe(periods=300, freq='d')

# Melakukan prediksi
forecast_final = final_model.predict(future)

# Menampilkan hasil prediksi
# print(forecast_final[['ds', 'yhat', 'yhat_lower', 'yhat_upper']])

# Menyimpan hasil prediksi ke dalam DataFrame
forecast_all_area_BT= forecast_final[['ds', 'yhat']]
forecast_all_area_BT.columns = ['Date', 'Forecast']

10:01:33 - cmdstanpy - INFO - Chain [1] start processing
10:01:34 - cmdstanpy - INFO - Chain [1] done processing
10:01:34 - cmdstanpy - INFO - Chain [1] start processing


SMAPE dengan changepoint_prior_scale=0.2, seasonality_prior_scale=0.5, n_changepoints=100, seasonality_mode=multiplicative: 24.39%


10:01:34 - cmdstanpy - INFO - Chain [1] done processing
10:01:35 - cmdstanpy - INFO - Chain [1] start processing


SMAPE dengan changepoint_prior_scale=0.2, seasonality_prior_scale=0.5, n_changepoints=150, seasonality_mode=multiplicative: 24.34%


10:01:35 - cmdstanpy - INFO - Chain [1] done processing


...

10:01:45 - cmdstanpy - INFO - Chain [1] done processing
10:01:46 - cmdstanpy - INFO - Chain [1] start processing


SMAPE dengan changepoint_prior_scale=0.5, seasonality_prior_scale=2, n_changepoints=100, seasonality_mode=multiplicative: 46.27%


10:01:47 - cmdstanpy - INFO - Chain [1] done processing
10:01:47 - cmdstanpy - INFO - Chain [1] start processing


SMAPE dengan changepoint_prior_scale=0.5, seasonality_prior_scale=2, n_changepoints=150, seasonality_mode=multiplicative: 47.77%

Parameter terbaik: changepoint_prior_scale=0.2, seasonality_prior_scale=0.5, n_changepoints=150, seasonality_mode=multiplicative
SMAPE terbaik: 24.34%


10:01:48 - cmdstanpy - INFO - Chain [1] done processing
10:01:48 - cmdstanpy - INFO - Chain [1] start processing
10:01:48 - cmdstanpy - INFO - Chain [1] done processing


In [55]:
# Pastikan kolom 'Date' dalam format datetime
forecast_all_area_BT['Date'] = pd.to_datetime(forecast_all_area_BT['Date']).dt.date
# Menambahkan data untuk 'Forecast'
forecast_rows = pd.DataFrame({
    'Tanggal': forecast_all_area_BT['Date'],
    'Tonase': forecast_all_area_BT['Forecast'].round(3),
    'Jenis': 'Forecast',
    'Kategori' : 'Coal',
    'Loc Src Area Name' : 'Banko Tengah'
})

df_production_actual_BT = pd.DataFrame({
    'Tanggal': pd.to_datetime(coal_all_area_BT['ds']).dt.date,
    'Tonase': coal_all_area_BT['y'].round(3),
    'Jenis': 'Actual',
    'Kategori' : 'Coal',
    'Loc Src Area Name' : 'Banko Tengah'
})
current_time = datetime.today().date()
best_smape_rounded = round(best_smape, 3)
smape_BT = pd.DataFrame({
    'Tanggal' : current_time,
    'Tonase' : [best_smape_rounded],
    'Jenis' :['Smape'],
    'Kategori' : ['Coal'],
    'Loc Src Area Name' : ['Banko Tengah']
})

# Menggabungkan semua DataFrame
result_df_BT = pd.concat([df_production_actual_BT,forecast_rows, smape_BT], ignore_index=True)
# result_df_BT


In [56]:
# Menggabungkan semua data frame
df_all_forecast = pd.concat([result_df_production, result_df_MTB, result_df_BB, result_df_BT, result_df_TAL], axis=0, ignore_index=True)
# df_all_forecast


In [58]:
df_temp_coal_nonreal = df[df['Jenis'].isin(['Plan'])]

In [59]:
df_temp_coal_nonreal = df_temp_coal_nonreal[df_temp_coal_nonreal['Product Type Name'].isin(['Market Brand','Mine Brand'])]


In [60]:
df_temp_coal_nonreal = df_temp_coal_nonreal[df_temp_coal_nonreal['Tanggal'] >= '2020-01-01']

In [61]:
df_temp_plan =  df_temp_coal_nonreal[['Tanggal','Tonase','Jenis','Loc Src Area Name']]

In [62]:
df_temp_all_area_plan = df_temp_plan.groupby(['Tanggal'])['Tonase'].sum().reset_index()

In [63]:
df_Plan_all = pd.DataFrame({
    'Tanggal': pd.to_datetime(df_temp_all_area_plan['Tanggal']).dt.date,
    'Tonase': df_temp_all_area_plan['Tonase'].round(3),
    'Jenis': 'Plan',
    'Kategori' : 'Coal',
    'Loc Src Area Name' : 'All Area'
})
df_Plan = pd.DataFrame({
    'Tanggal': pd.to_datetime(df_temp_plan['Tanggal']).dt.date,
    'Tonase': df_temp_plan['Tonase'].round(3),
    'Jenis': df_temp_plan['Jenis'],
    'Kategori' : 'Coal',
    'Loc Src Area Name' : df_temp_plan['Loc Src Area Name']
})

In [64]:
# OVERBURDEN
#  Preprocessing 
df_temp_OB_real = df[df['Product Type Name'].isin(['Waste'])].copy()


In [65]:
# preprocessing, mengabil nilai real tanpa plan (RKAP dan Rakor)
df_temp_OB_real = df_temp_OB_real[df_temp_OB_real['Jenis']=='Real']

In [75]:
# MTB
df_temp_OB_MTB = df_temp_OB_real[df_temp_OB_real['Loc Src Area Name']=='MTB']

In [76]:
temp_OB_MTB = df_temp_OB_MTB.groupby(['Tanggal'])['Tonase'].sum().reset_index()

In [77]:
temp_OB_MTB['Tanggal'] = pd.to_datetime(temp_OB_MTB['Tanggal'])


In [78]:
# Data yang diambil dari tanggl 1 januari 2020 (Sesuai Kesepakatan dengan Pihak PTBA)
df_prep_Date_MTB_OB = temp_OB_MTB[temp_OB_MTB['Tanggal'] >= '2020-01-01']

In [79]:
# reset index
df_prep_Date_MTB_OB = df_prep_Date_MTB_OB.reset_index().sort_values(by=['Tanggal'])
df_prep_Date_MTB_OB = df_prep_Date_MTB_OB[['Tanggal','Tonase']]

In [80]:
# Tidak Menyertakan Nilai 0 Dan null
temp_df_OB_MTB =  df_prep_Date_MTB_OB[df_prep_Date_MTB_OB['Tonase'] != 0].reset_index(drop=True).sort_values(by = ['Tanggal'])

In [81]:
OB_MTB = temp_df_OB_MTB[temp_df_OB_MTB['Tanggal'] <= pd.to_datetime('today')]

In [82]:
OB_MTB.columns=['ds','y']

In [83]:
# Fungsi SMAPE
def smape(actual, predicted):
    return 100 * np.mean(2 * np.abs(predicted - actual) / (np.abs(predicted) + np.abs(actual)))

# Data Preparation (Anda bisa menyesuaikan dengan dataset Anda)
train_size = int(0.9 * len(OB_MTB))
train_data = OB_MTB[:train_size]
test_data = OB_MTB[train_size:]

# Parameter tuning grid
changepoint_prior_scales = [ 0.1, 0.5]
seasonality_prior_scales = [ 1,5]
n_changepoints = [ 50, 100]
seasonality_modes = ['multiplicative']

# Kombinasi parameter
param_combinations = list(itertools.product(
    changepoint_prior_scales,
    seasonality_prior_scales,
    n_changepoints,
    seasonality_modes
))

# Variabel untuk menyimpan hasil tuning terbaik
best_smape = float('inf')
best_params = None

# Looping untuk setiap kombinasi parameter
for changepoint_prior_scale, seasonality_prior_scale, n_changepoints, seasonality_mode in param_combinations:
    
    # Inisialisasi model Prophet dengan parameter tuning
    model = Prophet(
        yearly_seasonality=True, 
        weekly_seasonality=False, 
        daily_seasonality=True,
        seasonality_mode=seasonality_mode,
        changepoint_prior_scale=changepoint_prior_scale,
        seasonality_prior_scale=seasonality_prior_scale,
        growth="linear",
        n_changepoints=n_changepoints,
        interval_width=0.95
    )
    
    # Menambahkan hari libur dan musiman tambahan jika diperlukan
    model.add_country_holidays(country_name='Indonesia')
    model.add_seasonality(name='monthly', period=30.5, fourier_order=5)

    # Melatih model dengan data train
    model.fit(train_data)

    # Membuat dataframe masa depan yang mencakup periode data test
    future = model.make_future_dataframe(periods=len(test_data), freq='D')

    # Melakukan prediksi
    forecast = model.predict(future)

    # Mengambil prediksi yang sesuai dengan data test
    forecast_test = forecast[-len(test_data):]

    # Menggabungkan data asli dengan prediksi
    test_data.loc[:, 'yhat'] = forecast_test['yhat'].values

    # Menghitung SMAPE untuk data test
    smape_value = smape(test_data['y'], test_data['yhat'])
    
    # Output hasil SMAPE
    print(f"SMAPE dengan changepoint_prior_scale={changepoint_prior_scale}, "
          f"seasonality_prior_scale={seasonality_prior_scale}, n_changepoints={n_changepoints}, "
          f"seasonality_mode={seasonality_mode}: {smape_value:.2f}%")

    # Menyimpan kombinasi parameter terbaik
    if smape_value < best_smape:
        best_smape = smape_value
        best_params = (changepoint_prior_scale, seasonality_prior_scale, n_changepoints, seasonality_mode)

# Menampilkan parameter terbaik
print(f"\nParameter terbaik: changepoint_prior_scale={best_params[0]}, "
      f"seasonality_prior_scale={best_params[1]}, n_changepoints={best_params[2]}, "
      f"seasonality_mode={best_params[3]}")
print(f"SMAPE terbaik: {best_smape:.2f}%")

# Perform cross-validation on the best model
best_model= Prophet(
    yearly_seasonality=True,
    weekly_seasonality=False,
    daily_seasonality=True,
    seasonality_mode=best_params[3],
    changepoint_prior_scale=best_params[0],
    seasonality_prior_scale=best_params[1],
    growth="linear",
    n_changepoints=best_params[2],
    interval_width=0.95
)
best_model.add_country_holidays(country_name='Indonesia')
best_model.add_seasonality(name='monthly', period=30.5, fourier_order=5)
best_model.fit(train_data)

# Latih model dengan seluruh data menggunakan parameter terbaik
final_model = Prophet(
    yearly_seasonality=True,
    weekly_seasonality=False,
    daily_seasonality=True,
    seasonality_mode=best_params[3],
    changepoint_prior_scale=best_params[0],
    seasonality_prior_scale=best_params[1],
    growth="linear",
    n_changepoints=best_params[2],
    interval_width=0.95
)
final_model.add_country_holidays(country_name='Indonesia')
final_model.add_seasonality(name='monthly', period=30.5, fourier_order=5)

# Melatih model dengan seluruh data
final_model.fit(OB_MTB)

# Membuat dataframe masa depan untuk 6 bulan ke depan
future = final_model.make_future_dataframe(periods=300, freq='d')

# Melakukan prediksi
forecast_final = final_model.predict(future)

# Menampilkan hasil prediksi
# print(forecast_final[['ds', 'yhat', 'yhat_lower', 'yhat_upper']])

# Menyimpan hasil prediksi ke dalam DataFrame
forecast_all_area_OB_MTB= forecast_final[['ds', 'yhat']]
forecast_all_area_OB_MTB.columns = ['Date', 'Forecast']

10:01:50 - cmdstanpy - INFO - Chain [1] start processing
10:01:51 - cmdstanpy - INFO - Chain [1] done processing


SMAPE dengan changepoint_prior_scale=0.1, seasonality_prior_scale=1, n_changepoints=50, seasonality_mode=multiplicative: 34.13%


10:01:52 - cmdstanpy - INFO - Chain [1] start processing
10:01:55 - cmdstanpy - INFO - Chain [1] done processing


SMAPE dengan changepoint_prior_scale=0.1, seasonality_prior_scale=1, n_changepoints=100, seasonality_mode=multiplicative: 34.23%


10:01:55 - cmdstanpy - INFO - Chain [1] start processing
10:01:57 - cmdstanpy - INFO - Chain [1] done processing


...

10:02:08 - cmdstanpy - INFO - Chain [1] start processing
10:02:09 - cmdstanpy - INFO - Chain [1] done processing


SMAPE dengan changepoint_prior_scale=0.5, seasonality_prior_scale=5, n_changepoints=50, seasonality_mode=multiplicative: 52.15%


10:02:10 - cmdstanpy - INFO - Chain [1] start processing
10:02:18 - cmdstanpy - INFO - Chain [1] done processing


SMAPE dengan changepoint_prior_scale=0.5, seasonality_prior_scale=5, n_changepoints=100, seasonality_mode=multiplicative: 46.90%

Parameter terbaik: changepoint_prior_scale=0.1, seasonality_prior_scale=1, n_changepoints=50, seasonality_mode=multiplicative
SMAPE terbaik: 34.13%


10:02:19 - cmdstanpy - INFO - Chain [1] start processing
10:02:22 - cmdstanpy - INFO - Chain [1] done processing
10:02:23 - cmdstanpy - INFO - Chain [1] start processing
10:02:26 - cmdstanpy - INFO - Chain [1] done processing


In [84]:
# Pastikan kolom 'Date' dalam format datetime
forecast_all_area_OB_MTB['Date'] = pd.to_datetime(forecast_all_area_OB_MTB['Date']).dt.date
# Menambahkan data untuk 'Forecast'
forecast_rows_OB_MTB = pd.DataFrame({
    'Tanggal': forecast_all_area_OB_MTB['Date'],
    'Tonase': forecast_all_area_OB_MTB['Forecast'].round(3),
    'Jenis': 'Forecast',
    'Kategori' : "Overburden",
    'Loc Src Area Name' : 'MTB'
})

df_temp_OB_MTB = pd.DataFrame({
    'Tanggal': pd.to_datetime(OB_MTB['ds']).dt.date,
    'Tonase': OB_MTB['y'].round(3),
    'Jenis': 'Actual',
    'Kategori' : "Overburden",
    'Loc Src Area Name' : 'MTB'
})
from datetime import datetime
current_time = datetime.today().date()
best_smape_rounded = round(best_smape, 3)
smape_all_OB_MTB = pd.DataFrame({
    'Tanggal' : current_time,
    'Tonase' : [best_smape_rounded],
    'Jenis' :['Smape'],
    'Kategori' : "Overburden",
    'Loc Src Area Name' : ['MTB']
})

# Menggabungkan semua DataFrame
result_df_OB_MTB = pd.concat([df_temp_OB_MTB,forecast_rows_OB_MTB, smape_all_OB_MTB], ignore_index=True)
# result_df_OB_MTB


In [85]:
# TAL
df_temp_OB_TAL = df_temp_OB_real[df_temp_OB_real['Loc Src Area Name']=='TAL']

In [86]:
temp_OB_TAL = df_temp_OB_TAL.groupby(['Tanggal'])['Tonase'].sum().reset_index()

In [87]:
temp_OB_TAL['Tanggal'] = pd.to_datetime(temp_OB_TAL['Tanggal'])

In [88]:
# Data yang diambil dari tanggl 1 januari 2020 (Sesuai Kesepakatan dengan Pihak PTBA)
df_prep_Date_TAL_OB = temp_OB_TAL[temp_OB_TAL['Tanggal'] >= '2020-01-01']

In [89]:
# reset index
df_prep_Date_index_TAL_OB = df_prep_Date_TAL_OB.reset_index().sort_values(by=['Tanggal'])
df_prep_Date_index_TAL_OB = df_prep_Date_index_TAL_OB[['Tanggal','Tonase']]

In [90]:
# Tidak Menyertakan Nilai 0 Dan null
temp_df_OB_TAL =  df_prep_Date_index_TAL_OB[df_prep_Date_index_TAL_OB['Tonase'] != 0].reset_index(drop=True).sort_values(by = ['Tanggal'])

In [91]:
OB_TAL = temp_df_OB_TAL[temp_df_OB_TAL['Tanggal'] <= pd.to_datetime('today')]

In [92]:
OB_TAL.columns=['ds','y']

In [93]:
# Fungsi SMAPE
def smape(actual, predicted):
    return 100 * np.mean(2 * np.abs(predicted - actual) / (np.abs(predicted) + np.abs(actual)))

# Data Preparation (Anda bisa menyesuaikan dengan dataset Anda)
train_size = int(0.9 * len(OB_TAL))
train_data = OB_TAL[:train_size]
test_data = OB_TAL[train_size:]

# Parameter tuning grid
changepoint_prior_scales = [ 0.05,0.1]
seasonality_prior_scales = [0.2,0.5 ]
n_changepoints = [ 20, 50]
seasonality_modes = ['multiplicative']

# Kombinasi parameter
param_combinations = list(itertools.product(
    changepoint_prior_scales,
    seasonality_prior_scales,
    n_changepoints,
    seasonality_modes
))

# Variabel untuk menyimpan hasil tuning terbaik
best_smape = float('inf')
best_params = None

# Looping untuk setiap kombinasi parameter
for changepoint_prior_scale, seasonality_prior_scale, n_changepoints, seasonality_mode in param_combinations:
    
    # Inisialisasi model Prophet dengan parameter tuning
    model = Prophet(
        yearly_seasonality=True, 
        weekly_seasonality=True, 
        daily_seasonality=True,
        seasonality_mode=seasonality_mode,
        changepoint_prior_scale=changepoint_prior_scale,
        seasonality_prior_scale=seasonality_prior_scale,
        growth="linear",
        n_changepoints=n_changepoints,
        interval_width=0.95

    )
    
    # Menambahkan hari libur dan musiman tambahan jika diperlukan
    model.add_country_holidays(country_name='Indonesia')
    model.add_seasonality(name='monthly', period=30.5, fourier_order=5)

    # Melatih model dengan data train
    model.fit(train_data)

    # Membuat dataframe masa depan yang mencakup periode data test
    future = model.make_future_dataframe(periods=len(test_data), freq='D')

    # Melakukan prediksi
    forecast = model.predict(future)

    # Mengambil prediksi yang sesuai dengan data test
    forecast_test = forecast[-len(test_data):]

    # Menggabungkan data asli dengan prediksi
    test_data.loc[:, 'yhat'] = forecast_test['yhat'].values

    # Menghitung SMAPE untuk data test
    smape_value = smape(test_data['y'], test_data['yhat'])
    
    # Output hasil SMAPE
    print(f"SMAPE dengan changepoint_prior_scale={changepoint_prior_scale}, "
          f"seasonality_prior_scale={seasonality_prior_scale}, n_changepoints={n_changepoints}, "
          f"seasonality_mode={seasonality_mode}: {smape_value:.2f}%")

    # Menyimpan kombinasi parameter terbaik
    if smape_value < best_smape:
        best_smape = smape_value
        best_params = (changepoint_prior_scale, seasonality_prior_scale, n_changepoints, seasonality_mode)

# Menampilkan parameter terbaik
print(f"\nParameter terbaik: changepoint_prior_scale={best_params[0]}, "
      f"seasonality_prior_scale={best_params[1]}, n_changepoints={best_params[2]}, "
      f"seasonality_mode={best_params[3]}")
print(f"SMAPE terbaik: {best_smape:.2f}%")

# Perform cross-validation on the best model
best_model = Prophet(
    yearly_seasonality=True,
    weekly_seasonality=False,
    daily_seasonality=True,
    seasonality_mode=best_params[3],
    changepoint_prior_scale=best_params[0],
    seasonality_prior_scale=best_params[1],
    growth="linear",
    n_changepoints=best_params[2],
    interval_width=0.95
)
best_model.add_country_holidays(country_name='Indonesia')
best_model.add_seasonality(name='monthly', period=30.5, fourier_order=5)
best_model.fit(train_data)

# Latih model dengan seluruh data menggunakan parameter terbaik
final_model = Prophet(
    yearly_seasonality=True,
    weekly_seasonality=True,
    daily_seasonality=True,
    seasonality_mode=best_params[3],
    changepoint_prior_scale=best_params[0],
    seasonality_prior_scale=best_params[1],
    growth="linear",
    n_changepoints=best_params[2],
    interval_width=0.95
)
final_model.add_country_holidays(country_name='Indonesia')
final_model.add_seasonality(name='monthly', period=30.5, fourier_order=5)

# Melatih model dengan seluruh data
final_model.fit(OB_TAL)

# Membuat dataframe masa depan untuk 6 bulan ke depan
future = final_model.make_future_dataframe(periods=300, freq='d')

# Melakukan prediksi
forecast_final = final_model.predict(future)

# Menampilkan hasil prediksi
# print(forecast_final[['ds', 'yhat', 'yhat_lower', 'yhat_upper']])

# Menyimpan hasil prediksi ke dalam DataFrame
forecast_all_area_OB_TAL= forecast_final[['ds', 'yhat']]
forecast_all_area_OB_TAL.columns = ['Date', 'Forecast']


10:02:27 - cmdstanpy - INFO - Chain [1] start processing
10:02:27 - cmdstanpy - INFO - Chain [1] done processing


SMAPE dengan changepoint_prior_scale=0.05, seasonality_prior_scale=0.2, n_changepoints=20, seasonality_mode=multiplicative: 25.73%


10:02:28 - cmdstanpy - INFO - Chain [1] start processing
10:02:28 - cmdstanpy - INFO - Chain [1] done processing


SMAPE dengan changepoint_prior_scale=0.05, seasonality_prior_scale=0.2, n_changepoints=50, seasonality_mode=multiplicative: 26.16%


10:02:29 - cmdstanpy - INFO - Chain [1] start processing
10:02:29 - cmdstanpy - INFO - Chain [1] done processing


...

10:02:32 - cmdstanpy - INFO - Chain [1] start processing
10:02:33 - cmdstanpy - INFO - Chain [1] done processing


SMAPE dengan changepoint_prior_scale=0.1, seasonality_prior_scale=0.5, n_changepoints=20, seasonality_mode=multiplicative: 28.88%


10:02:33 - cmdstanpy - INFO - Chain [1] start processing
10:02:34 - cmdstanpy - INFO - Chain [1] done processing


SMAPE dengan changepoint_prior_scale=0.1, seasonality_prior_scale=0.5, n_changepoints=50, seasonality_mode=multiplicative: 28.46%

Parameter terbaik: changepoint_prior_scale=0.1, seasonality_prior_scale=0.2, n_changepoints=20, seasonality_mode=multiplicative
SMAPE terbaik: 24.36%


10:02:34 - cmdstanpy - INFO - Chain [1] start processing
10:02:35 - cmdstanpy - INFO - Chain [1] done processing
10:02:35 - cmdstanpy - INFO - Chain [1] start processing
10:02:35 - cmdstanpy - INFO - Chain [1] done processing


In [94]:
# Pastikan kolom 'Date' dalam format datetime
forecast_all_area_OB_TAL['Date'] = pd.to_datetime(forecast_all_area_OB_TAL['Date']).dt.date
# Menambahkan data untuk 'Forecast'
forecast_rows_OB_TAL = pd.DataFrame({
    'Tanggal': forecast_all_area_OB_TAL['Date'],
    'Tonase': forecast_all_area_OB_TAL['Forecast'].round(3),
    'Jenis': 'Forecast',
    'Kategori' : "Overburden",
    'Loc Src Area Name' : 'TAL'
})

df_temp_OB_TAL = pd.DataFrame({
    'Tanggal': pd.to_datetime(OB_TAL['ds']).dt.date,
    'Tonase': OB_TAL['y'].round(3),
    'Jenis': 'Actual',
    'Kategori' : "Overburden",
    'Loc Src Area Name' : 'TAL'
})
from datetime import datetime
current_time = datetime.today().date()
best_smape_rounded = round(best_smape, 3)
smape_all_OB_TAL = pd.DataFrame({
    'Tanggal' : current_time,
    'Tonase' : [best_smape_rounded],
    'Jenis' :['Smape'],
    'Kategori' : "Overburden",
    'Loc Src Area Name' : ['TAL']
})

# Menggabungkan semua DataFrame
result_df_OB_TAL = pd.concat([df_temp_OB_TAL,forecast_rows_OB_TAL, smape_all_OB_TAL], ignore_index=True)
# result_df_OB_TAL


In [95]:
# Banko Barat

df_temp_OB_BB = df_temp_OB_real[df_temp_OB_real['Loc Src Area Name']=='Banko Barat']

In [96]:
df_temp_OB_BB = df_temp_OB_BB.groupby(['Tanggal'])['Tonase'].sum().reset_index()

In [97]:
df_temp_OB_BB['Tanggal'] = pd.to_datetime(df_temp_OB_BB['Tanggal'])

In [98]:
# Data yang diambil dari tanggl 1 januari 2020 (Sesuai Kesepakatan dengan Pihak PTBA)
df_prep_Date_BB_OB = df_temp_OB_BB[df_temp_OB_BB['Tanggal'] >= '2020-01-01']

In [99]:
# reset index
df_prep_Date_index_BB_OB = df_prep_Date_BB_OB.reset_index().sort_values(by=['Tanggal'])
df_prep_Date_index_BB_OB = df_prep_Date_index_BB_OB[['Tanggal','Tonase']]


In [100]:
# Tidak Menyertakan Nilai 0 Dan null
OB_BB =  df_prep_Date_index_BB_OB[df_prep_Date_index_BB_OB['Tonase'] != 0].reset_index(drop=True).sort_values(by = ['Tanggal'])

In [101]:
OB_BB.columns=['ds','y']

In [102]:
# Fungsi SMAPE
def smape(actual, predicted):
    return 100 * np.mean(2 * np.abs(predicted - actual) / (np.abs(predicted) + np.abs(actual)))

# Data Preparation (Anda bisa menyesuaikan dengan dataset Anda)
train_size = int(0.9 * len(OB_BB))
train_data = OB_BB[:train_size]
test_data = OB_BB[train_size:]

# Parameter tuning grid
changepoint_prior_scales = [ 1, 2]
seasonality_prior_scales = [0.5, 1]
n_changepoints = [ 50,100]
seasonality_modes = ['additive']

# Kombinasi parameter
param_combinations = list(itertools.product(
    changepoint_prior_scales,
    seasonality_prior_scales,
    n_changepoints,
    seasonality_modes
))

# Variabel untuk menyimpan hasil tuning terbaik
best_smape = float('inf')
best_params = None

# Looping untuk setiap kombinasi parameter
for changepoint_prior_scale, seasonality_prior_scale, n_changepoints, seasonality_mode in param_combinations:
    
    # Inisialisasi model Prophet dengan parameter tuning
    model = Prophet(
        yearly_seasonality=True, 
        weekly_seasonality=True, 
        daily_seasonality=True,
        seasonality_mode=seasonality_mode,
        changepoint_prior_scale=changepoint_prior_scale,
        seasonality_prior_scale=seasonality_prior_scale,
        growth="flat",
        n_changepoints=n_changepoints,
        interval_width=0.95

    )
    
    # Menambahkan hari libur dan musiman tambahan jika diperlukan
    model.add_country_holidays(country_name='Indonesia')
    model.add_seasonality(name='monthly', period=30.5, fourier_order=5)

    # Melatih model dengan data train
    model.fit(train_data)

    # Membuat dataframe masa depan yang mencakup periode data test
    future = model.make_future_dataframe(periods=len(test_data), freq='D')

    # Melakukan prediksi
    forecast = model.predict(future)

    # Mengambil prediksi yang sesuai dengan data test
    forecast_test = forecast[-len(test_data):]

    # Menggabungkan data asli dengan prediksi
    test_data.loc[:, 'yhat'] = forecast_test['yhat'].values

    # Menghitung SMAPE untuk data test
    smape_value = smape(test_data['y'], test_data['yhat'])
    
    # Output hasil SMAPE
    print(f"SMAPE dengan changepoint_prior_scale={changepoint_prior_scale}, "
          f"seasonality_prior_scale={seasonality_prior_scale}, n_changepoints={n_changepoints}, "
          f"seasonality_mode={seasonality_mode}: {smape_value:.2f}%")

    # Menyimpan kombinasi parameter terbaik
    if smape_value < best_smape:
        best_smape = smape_value
        best_params = (changepoint_prior_scale, seasonality_prior_scale, n_changepoints, seasonality_mode)

# Menampilkan parameter terbaik
print(f"\nParameter terbaik: changepoint_prior_scale={best_params[0]}, "
      f"seasonality_prior_scale={best_params[1]}, n_changepoints={best_params[2]}, "
      f"seasonality_mode={best_params[3]}")
print(f"SMAPE terbaik: {best_smape:.2f}%")

# Perform cross-validation on the best model
best_model = Prophet(
    yearly_seasonality=True,
    weekly_seasonality=False,
    daily_seasonality=True,
    seasonality_mode=best_params[3],
    changepoint_prior_scale=best_params[0],
    seasonality_prior_scale=best_params[1],
    growth="flat",
    n_changepoints=best_params[2],
    interval_width=0.95
)
best_model.add_country_holidays(country_name='Indonesia')
best_model.add_seasonality(name='monthly', period=30.5, fourier_order=5)
best_model.fit(train_data)

# Latih model dengan seluruh data menggunakan parameter terbaik
final_model = Prophet(
    yearly_seasonality=True,
    weekly_seasonality=True,
    daily_seasonality=True,
    seasonality_mode=best_params[3],
    changepoint_prior_scale=best_params[0],
    seasonality_prior_scale=best_params[1],
    growth="flat",
    n_changepoints=best_params[2],
    interval_width=0.95
)
final_model.add_country_holidays(country_name='Indonesia')
final_model.add_seasonality(name='monthly', period=30.5, fourier_order=5)

# Melatih model dengan seluruh data
final_model.fit(OB_BB)

# Membuat dataframe masa depan untuk 6 bulan ke depan
future = final_model.make_future_dataframe(periods=300, freq='d')

# Melakukan prediksi
forecast_final = final_model.predict(future)

# Menampilkan hasil prediksi
# print(forecast_final[['ds', 'yhat', 'yhat_lower', 'yhat_upper']])

# Menyimpan hasil prediksi ke dalam DataFrame
forecast_all_area_OB_BB= forecast_final[['ds', 'yhat']]
forecast_all_area_OB_BB.columns = ['Date', 'Forecast']

10:02:36 - cmdstanpy - INFO - Chain [1] start processing
10:02:36 - cmdstanpy - INFO - Chain [1] done processing


SMAPE dengan changepoint_prior_scale=1, seasonality_prior_scale=0.5, n_changepoints=50, seasonality_mode=additive: 19.42%


10:02:37 - cmdstanpy - INFO - Chain [1] start processing
10:02:37 - cmdstanpy - INFO - Chain [1] done processing


SMAPE dengan changepoint_prior_scale=1, seasonality_prior_scale=0.5, n_changepoints=100, seasonality_mode=additive: 19.42%


10:02:37 - cmdstanpy - INFO - Chain [1] start processing
10:02:38 - cmdstanpy - INFO - Chain [1] done processing


...

10:02:40 - cmdstanpy - INFO - Chain [1] start processing
10:02:40 - cmdstanpy - INFO - Chain [1] done processing


SMAPE dengan changepoint_prior_scale=2, seasonality_prior_scale=1, n_changepoints=50, seasonality_mode=additive: 19.42%


10:02:40 - cmdstanpy - INFO - Chain [1] start processing
10:02:41 - cmdstanpy - INFO - Chain [1] done processing


SMAPE dengan changepoint_prior_scale=2, seasonality_prior_scale=1, n_changepoints=100, seasonality_mode=additive: 19.42%

Parameter terbaik: changepoint_prior_scale=1, seasonality_prior_scale=1, n_changepoints=100, seasonality_mode=additive
SMAPE terbaik: 19.42%


10:02:41 - cmdstanpy - INFO - Chain [1] start processing
10:02:41 - cmdstanpy - INFO - Chain [1] done processing
10:02:41 - cmdstanpy - INFO - Chain [1] start processing
10:02:42 - cmdstanpy - INFO - Chain [1] done processing


In [103]:
# Pastikan kolom 'Date' dalam format datetime
forecast_all_area_OB_BB['Date'] = pd.to_datetime(forecast_all_area_OB_BB['Date']).dt.date
# Menambahkan data untuk 'Forecast'
forecast_rows_OB_BB = pd.DataFrame({
    'Tanggal': forecast_all_area_OB_BB['Date'],
    'Tonase': forecast_all_area_OB_BB['Forecast'].round(3),
    'Jenis': 'Forecast',
    'Kategori' :  'Overburden',
    'Loc Src Area Name' : 'Banko Barat'
})

df_temp_OB_BB = pd.DataFrame({
    'Tanggal': pd.to_datetime(OB_BB['ds']).dt.date,
    'Tonase': OB_BB['y'].round(3),
    'Jenis': 'Actual',
    'Kategori' :  'Overburden',
    'Loc Src Area Name' : 'Banko Barat'
})
from datetime import datetime
current_time = datetime.today().date()
best_smape_rounded = round(best_smape, 3)
smape_all_OB_BB = pd.DataFrame({
    'Tanggal' : current_time,
    'Tonase' : [best_smape_rounded],
    'Jenis' :['Smape'],
    'Kategori': ['Overburden'],
    'Loc Src Area Name' : ['Banko Barat']
})

# Menggabungkan semua DataFrame
result_df_OB_BB = pd.concat([df_temp_OB_BB,forecast_rows_OB_BB, smape_all_OB_BB], ignore_index=True)
# result_df_OB_BB


In [104]:
# Banko Barat
df_temp_OB_BT = df_temp_OB_real[df_temp_OB_real['Loc Src Area Name']=='Banko Tengah']

In [105]:
df_temp_OB_BT = df_temp_OB_BT.groupby(['Tanggal'])['Tonase'].sum().reset_index()

In [106]:
df_temp_OB_BT['Tanggal'] = pd.to_datetime(df_temp_OB_BT['Tanggal'])

In [107]:
# # Data yang diambil dari tanggl 1 januari 2020 (Sesuai Kesepakatan dengan Pihak PTBA)
df_prep_Date_BT_OB = df_temp_OB_BT[df_temp_OB_BT['Tanggal'] >= '2020-01-01']

In [108]:
# reset index
df_prep_Date_index_BT_OB = df_prep_Date_BT_OB.reset_index().sort_values(by=['Tanggal'])
df_prep_Date_index_BT_OB = df_prep_Date_index_BT_OB[['Tanggal','Tonase']]

In [109]:
# Tidak Menyertakan Nilai 0 Dan null
OB_BT =  df_prep_Date_index_BT_OB[df_prep_Date_index_BT_OB['Tonase'] != 0].reset_index(drop=True).sort_values(by = ['Tanggal'])

In [110]:
OB_BT.columns=['ds','y']

In [111]:
# Fungsi SMAPE
def smape(actual, predicted):
    return 100 * np.mean(2 * np.abs(predicted - actual) / (np.abs(predicted) + np.abs(actual)))

# Data Preparation (Anda bisa menyesuaikan dengan dataset Anda)
train_size = int(0.9 * len(OB_BT))
train_data = OB_BT[:train_size]
test_data = OB_BT[train_size:]

# Parameter tuning grid
changepoint_prior_scales = [ 0.5, 1]
seasonality_prior_scales = [0.5, 1]
n_changepoints = [ 50,100]
seasonality_modes = ['multiplicative']

# Kombinasi parameter
param_combinations = list(itertools.product(
    changepoint_prior_scales,
    seasonality_prior_scales,
    n_changepoints,
    seasonality_modes
))

# Variabel untuk menyimpan hasil tuning terbaik
best_smape = float('inf')
best_params = None

# Looping untuk setiap kombinasi parameter
for changepoint_prior_scale, seasonality_prior_scale, n_changepoints, seasonality_mode in param_combinations:
    
    # Inisialisasi model Prophet dengan parameter tuning
    model = Prophet(
        yearly_seasonality=True, 
        weekly_seasonality=True, 
        daily_seasonality=True,
        seasonality_mode=seasonality_mode,
        changepoint_prior_scale=changepoint_prior_scale,
        seasonality_prior_scale=seasonality_prior_scale,
        growth="linear",
        n_changepoints=n_changepoints,
        interval_width=0.95
    )
    
    # Menambahkan hari libur dan musiman tambahan jika diperlukan
    model.add_country_holidays(country_name='Indonesia')
    model.add_seasonality(name='monthly', period=30.5, fourier_order=5)

    # Melatih model dengan data train
    model.fit(train_data)

    # Membuat dataframe masa depan yang mencakup periode data test
    future = model.make_future_dataframe(periods=len(test_data), freq='D')

    # Melakukan prediksi
    forecast = model.predict(future)

    # Mengambil prediksi yang sesuai dengan data test
    forecast_test = forecast[-len(test_data):]

    # Menggabungkan data asli dengan prediksi
    test_data.loc[:, 'yhat'] = forecast_test['yhat'].values

    # Menghitung SMAPE untuk data test
    smape_value = smape(test_data['y'], test_data['yhat'])
    
    # Output hasil SMAPE
    print(f"SMAPE dengan changepoint_prior_scale={changepoint_prior_scale}, "
          f"seasonality_prior_scale={seasonality_prior_scale}, n_changepoints={n_changepoints}, "
          f"seasonality_mode={seasonality_mode}: {smape_value:.2f}%")

    # Menyimpan kombinasi parameter terbaik
    if smape_value < best_smape:
        best_smape = smape_value
        best_params = (changepoint_prior_scale, seasonality_prior_scale, n_changepoints, seasonality_mode)

# Menampilkan parameter terbaik
print(f"\nParameter terbaik: changepoint_prior_scale={best_params[0]}, "
      f"seasonality_prior_scale={best_params[1]}, n_changepoints={best_params[2]}, "
      f"seasonality_mode={best_params[3]}")
print(f"SMAPE terbaik: {best_smape:.2f}%")

# Perform cross-validation on the best model
best_model = Prophet(
    yearly_seasonality=True,
    weekly_seasonality=False,
    daily_seasonality=True,
    seasonality_mode=best_params[3],
    changepoint_prior_scale=best_params[0],
    seasonality_prior_scale=best_params[1],
    growth="linear",
    n_changepoints=best_params[2],
    interval_width=0.95
)
best_model.add_country_holidays(country_name='Indonesia')
best_model.add_seasonality(name='monthly', period=30.5, fourier_order=5)
best_model.fit(train_data)

# Latih model dengan seluruh data menggunakan parameter terbaik
final_model = Prophet(
    yearly_seasonality=True,
    weekly_seasonality=True,
    daily_seasonality=True,
    seasonality_mode=best_params[3],
    changepoint_prior_scale=best_params[0],
    seasonality_prior_scale=best_params[1],
    growth="linear",
    n_changepoints=best_params[2],
    interval_width=0.95
)
final_model.add_country_holidays(country_name='Indonesia')
final_model.add_seasonality(name='monthly', period=30.5, fourier_order=5)

# Melatih model dengan seluruh data
final_model.fit(OB_BT)

# Membuat dataframe masa depan untuk 6 bulan ke depan
future = final_model.make_future_dataframe(periods=300, freq='d')

# Melakukan prediksi
forecast_final = final_model.predict(future)

# Menampilkan hasil prediksi
# print(forecast_final[['ds', 'yhat', 'yhat_lower', 'yhat_upper']])

# Menyimpan hasil prediksi ke dalam DataFrame
forecast_all_area_OB_BT= forecast_final[['ds', 'yhat']]
forecast_all_area_OB_BT.columns = ['Date', 'Forecast']

10:02:42 - cmdstanpy - INFO - Chain [1] start processing
10:02:43 - cmdstanpy - INFO - Chain [1] done processing


SMAPE dengan changepoint_prior_scale=0.5, seasonality_prior_scale=0.5, n_changepoints=50, seasonality_mode=multiplicative: 31.98%


10:02:43 - cmdstanpy - INFO - Chain [1] start processing
10:02:44 - cmdstanpy - INFO - Chain [1] done processing
10:02:44 - cmdstanpy - INFO - Chain [1] start processing


SMAPE dengan changepoint_prior_scale=0.5, seasonality_prior_scale=0.5, n_changepoints=100, seasonality_mode=multiplicative: 27.97%


10:02:45 - cmdstanpy - INFO - Chain [1] done processing
10:02:45 - cmdstanpy - INFO - Chain [1] start processing


...

10:02:49 - cmdstanpy - INFO - Chain [1] start processing
10:02:50 - cmdstanpy - INFO - Chain [1] done processing
10:02:51 - cmdstanpy - INFO - Chain [1] start processing


SMAPE dengan changepoint_prior_scale=1, seasonality_prior_scale=1, n_changepoints=50, seasonality_mode=multiplicative: 19.51%


10:02:53 - cmdstanpy - INFO - Chain [1] done processing


SMAPE dengan changepoint_prior_scale=1, seasonality_prior_scale=1, n_changepoints=100, seasonality_mode=multiplicative: 20.68%

Parameter terbaik: changepoint_prior_scale=1, seasonality_prior_scale=1, n_changepoints=50, seasonality_mode=multiplicative
SMAPE terbaik: 19.51%


10:02:53 - cmdstanpy - INFO - Chain [1] start processing
10:02:54 - cmdstanpy - INFO - Chain [1] done processing
10:02:54 - cmdstanpy - INFO - Chain [1] start processing
10:02:55 - cmdstanpy - INFO - Chain [1] done processing


In [112]:
# Pastikan kolom 'Date' dalam format datetime
forecast_all_area_OB_BT['Date'] = pd.to_datetime(forecast_all_area_OB_BT['Date']).dt.date
# Menambahkan data untuk 'Forecast'
forecast_rows_OB_BT = pd.DataFrame({
    'Tanggal': forecast_all_area_OB_BT['Date'],
    'Tonase': forecast_all_area_OB_BT['Forecast'].round(3),
    'Jenis': 'Forecast',
    'Kategori' : "Overburden",
    'Loc Src Area Name' : 'Banko Tengah'
})

df_temp_OB_BT = pd.DataFrame({
    'Tanggal': pd.to_datetime(OB_BT['ds']).dt.date,
    'Tonase': OB_BT['y'].round(3),
    'Jenis': 'Actual',
    'Kategori' : "Overburden",
    'Loc Src Area Name' : 'Banko Tengah'
})
from datetime import datetime
current_time = datetime.today().date()
best_smape_rounded = round(best_smape, 3)
smape_all_OB_BT = pd.DataFrame({
    'Tanggal' : current_time,
    'Tonase' : [best_smape_rounded],
    'Jenis' :['Smape'],
    'Kategori' : ["Overburden"],
    'Loc Src Area Name' : ['Banko Tengah']
})

# Menggabungkan semua DataFrame
result_df_OB_BT = pd.concat([df_temp_OB_BT,forecast_rows_OB_BT, smape_all_OB_BT], ignore_index=True)
# result_df_OB_BT


In [113]:
df_temp_OB_nonreal = df[df['Jenis'].isin(['Plan'])]
df_temp_OB_nonreal = df_temp_OB_nonreal[df_temp_OB_nonreal['Product Type Name'].isin(['Waste'])]
# df_temp_OB_nonreal
df_temp_OB_nonreal = df_temp_OB_nonreal[df_temp_OB_nonreal['Tanggal'] >= '2020-01-01']
# df_temp_OB_nonreal
df_temp_plan =  df_temp_OB_nonreal[['Tanggal','Tonase','Jenis','Loc Src Area Name']]
# df_temp_plan
df_temp_all_area_plan = df_temp_plan.groupby(['Tanggal'])['Tonase'].sum().reset_index()
# df_temp_all_area_plan
df_Plan_all_OB = pd.DataFrame({
    'Tanggal': pd.to_datetime(df_temp_all_area_plan['Tanggal']).dt.date,
    'Tonase': df_temp_all_area_plan['Tonase'].round(3),
    'Jenis': 'Plan',
    'Kategori' : 'Overburden',
    'Loc Src Area Name' : 'All Area'
})
df_Plan_OB = pd.DataFrame({
    'Tanggal': pd.to_datetime(df_temp_plan['Tanggal']).dt.date,
    'Tonase': df_temp_plan['Tonase'].round(3),
    'Jenis': df_temp_plan['Jenis'],
    'Kategori' : 'Overburden',
    'Loc Src Area Name' : df_temp_plan['Loc Src Area Name']
})

In [114]:
Df_Final =  pd.concat([result_df_production, result_df_MTB, result_df_BB, result_df_BT, result_df_TAL,
                       df_Plan_all,df_Plan, result_df_OB_BT,result_df_OB_BB, result_df_OB_MTB, 
                       result_df_OB_TAL,df_Plan_all_OB,df_Plan_OB],axis=0, ignore_index=False)
Df_Final

,Tanggal,Tonase,Jenis,Kategori,Loc Src Area Name
0,2020-01-01,24284.683,Actual,Coal,All Area
1,2020-01-02,59932.859,Actual,Coal,All Area
2,2020-01-03,46403.077,Actual,Coal,All Area
3,2020-01-04,68253.579,Actual,Coal,All Area
4,2020-01-05,51894.829,Actual,Coal,All Area
...,...,...,...,...,...
254026,2022-09-02,33333.333,Plan,Overburden,TAL
254043,2023-01-26,11612.903,Plan,Overburden,Banko Barat
254054,2021-10-19,40967.742,Plan,Overburden,Banko Barat
254060,2021-09-20,35500.000,Plan,Overburden,TAL


In [115]:
Df_Final['Tanggal'] = pd.to_datetime(Df_Final['Tanggal'],format='mixed', errors='coerce')
Df_Final['Last Updated'] = pd.Timestamp.now()
Df_Final.columns=['tanggal','tonase','jenis','kategori','loc_src_Area_Name','last_update']

In [116]:
new_column_order = ['tanggal','jenis','kategori','loc_src_Area_Name','tonase','last_update']
Df_Final = Df_Final[new_column_order]

In [ ]:
# Df_Final.to_csv('Df_Snapshot.csv')

In [119]:
print("Sedang Import ke DM_forecast_prod_v1")

import pyodbc
from dotenv import load_dotenv
import os
import pandas as pd

# Koneksi ke SQL Server
server = '10.3.4.99,1433'  # Ubah sesuai kebutuhan
database = 'dwh_dev'
username = 'cybertrend'
password = 'cn70HJBQHRfsoOS'

# Inisialisasi koneksi ke SQL Server
try:
    conn = pyodbc.connect(
        f'DRIVER={{SQL Server}};SERVER={server};DATABASE={database};UID={username};PWD={password}'
    )
    print("Koneksi ke database berhasil.")
except Exception as e:
    print(f"Error saat mencoba koneksi ke database: {e}")
    exit()

cursor = conn.cursor()

# Proses truncate tabel
truncate_query = "TRUNCATE TABLE dwh.DM_forecast_prod_v1"
try:
    cursor.execute(truncate_query)
    conn.commit()
    print("Tabel berhasil di-truncate.")
except Exception as e:
    print(f"Error saat truncate tabel: {e}")
    conn.close()
    exit()


# Pastikan `Df_Final` sudah didefinisikan
if 'Df_Final' not in globals():
    print("Error: DataFrame `Df_Final` tidak ditemukan.")
    conn.close()
    exit()

# Menyiapkan query untuk insert
insert_query = """
    INSERT INTO dwh.DM_forecast_prod_v1 
    (tanggal, jenis, kategori, loc_src_Area_Name, tonase, last_update)
    VALUES (?, ?, ?, ?, ?, ?)
"""

rows_to_insert = [tuple(row) for row in Df_Final.itertuples(index=False)]

# Proses insert data
cursor.fast_executemany = True
try:
    cursor.executemany(insert_query, rows_to_insert)
    conn.commit()
    print(f"Berhasil Insert {len(rows_to_insert)} baris data.")
except Exception as e:
    conn.rollback()
    print(f"Error saat menginsert data: {e}")

# Menutup koneksi
cursor.close()
conn.close()
print("Forecasting Done")


Sedang Import ke DM_forecast_prod_v1
Koneksi ke database berhasil.
Tabel berhasil di-truncate.
Berhasil Insert 149320 baris data.
Forecasting Done
